## 🔎 Agent Observability - Logs, Traces & Metrics

**Day 4A — Google ADK | Local VS Code**

Today, you'll learn:

- How to add observability to the agent you've built and
- How to evaluate if the agents are working as expected
---

In this notebook, you'll:

- Set up logging configuration
- Create a broken agent. Use adk web UI & logs to identify exactly why the agent fails
- Understand how to implement logging in production
- Learn when to use built-in logging vs custom solutions

**No Kaggle. Just code.**

### What is Agent Observability?
🚨 The challenge: Unlike traditional software that fails predictably, AI agents can fail mysteriously. Example:

- User: "Find quantum computing papers"
- Agent: "I cannot help with that request."
- You: 😭 WHY?? Is it the prompt? Missing tools? API error?

💡 The Solution: Agent observability gives you complete visibility into your agent's decision-making process. You'll see exactly what prompts are sent to the LLM, which tools are available, how the model responds, and where failures occur.

- DEBUG Log: LLM Request shows "Functions: []" (no tools!)
- You: 🎯 Aha! Missing google_search tool - easy fix!

Foundational pillars of Agent Observability
- Logs: A log is a record of a single event, telling you what happened at a specific moment.
- Traces: A trace connects the logs into a single story, showing you why a final result occurred by revealing the entire sequence of steps.
- Metrics: Metrics are the summary numbers (like averages and error rates) that tell you how well the agent is performing overall.

### Load API Key

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("Add GOOGLE_API_KEY to .env in project root!")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"

print("Gemini API key loaded!")

Gemini API key loaded!


### Set up logging and cleanup old files
Let's configure logging for our debugging session. The following cell makes sure we also capture other log levels, like DEBUG.

In [2]:
import logging
import os
import time
from logging import FileHandler

# -------------------------------------------------
# 1. Clean-up with retry (Windows-friendly)
# -------------------------------------------------
log_files = ["logger.log", "web.log", "tunnel.log"]

for log_file in log_files:
    if os.path.exists(log_file):
        for attempt in range(5):                     # try up to 5 times
            try:
                os.remove(log_file)
                print(f"Cleaned up {log_file}")
                break
            except PermissionError:
                print(f"  ↻ {log_file} locked, retry {attempt+1}/5 …")
                time.sleep(1)                       # give Windows a moment
            except Exception as e:
                print(f"  Could not delete {log_file}: {e}")
                break
        else:
            print(f"  Could not clean {log_file} – will reuse it")

# -------------------------------------------------
# 2. Configure logging (force reset + safe handler)
# -------------------------------------------------
logging.basicConfig(
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)d %(levelname)s: %(message)s",
    handlers=[
        FileHandler("logger.log", mode="a"),   # append if file already exists
        logging.StreamHandler()                # also print to notebook output
    ],
    force=True                                 # <-- IMPORTANT on Windows
)

print("Logging configured – ready for DEBUG output")

Cleaned up logger.log
Logging configured – ready for DEBUG output


### 🐞 Section 2: Hands-On Debugging
#### 2.1: Create a "Research Paper Finder" Agent
Our goal: Build a research paper finder agent that helps users find academic papers on any topic.

But first, let's intentionally create an incorrect version of the agent to practice debugging!

### Agent definition
Next, let's create our root agent.

We'll configure it as an LlmAgent, give it a name, model and instruction.
The root_agent gets the user prompt and delegates the search to the google_search_agent.
Then, the agent uses the count_papers tool to count the number of papers returned.

In [3]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

# ---- Intentionally pass incorrect datatype - `str` instead of `List[str]` ----
def count_papers(papers: str):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)


# Google Search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)


# Root agent
root_agent = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.
    """,
    tools=[AgentTool(agent=google_search_agent), count_papers]
)

In [4]:
print("Running broken agent...\n")
try:
    response = root_agent.run("Find research papers on quantum computing")
    print("Success:", response)
except Exception as e:
    print("Failed as expected:", str(e))

Running broken agent...

Failed as expected: 'LlmAgent' object has no attribute 'run'


In [6]:
# Check the DEBUG logs from the broken agent
print("🔍 Examining logs for debugging clues...\n")
!type logger.log

🔍 Examining logs for debugging clues...



### 🧑‍💻 Section 3: Logging in production
🎯 Great! You can now debug agent failures using ADK web UI and DEBUG logs.

But what happens when you move beyond development? Real-world scenarios where you need to move beyond the web UI:

❌ Problem 1: Production Deployment

- You: "Let me open the ADK web UI to check why the agent failed"
- DevOps: "Um... this is a production server. No web UI access."
- You: 😱 "How do I debug production issues?"
❌ Problem 2: Automated Systems

- You: "The agent runs 1000 times per day in our pipeline"
- Boss: "Which runs are slow? What's our success rate?"
- You: 😰 "I'd have to manually check the web UI 1000 times..."
💡 The Solution:

We need a way to capture observability data or in other words, add logs to our code.

👉 In traditional software development, this is done by adding log statements in Python functions - and agents are no different! We need to add log statements to our agent and a common approach is to add log statements to Plugins.

#### 3.1: How to add logs for production observability?
A Plugin is a custom code module that runs automatically at various stages of your agent's lifecycle. Plugins are composed of "Callbacks" which provide the hooks to interrupt an agent's flow. Think of it like this:

Your agent workflow: User message → Agent thinks → Calls tools → Returns response
Plugin hooks into this: Before agent starts → After tool runs → When LLM responds → etc.
Plugin contains your custom code: Logging, monitoring, security checks, caching, etc.

### Callbacks
Callbacks are the atomic components inside a Plugin - these are just Python functions that run at specific points in an agent's lifecycle! Callbacks are grouped together to create a Plugin.

There are different kinds of callbacks such as:

before/after_agent_callbacks - runs before/after an agent is invoked
before/after_tool_callbacks - runs before/after a tool is called
before/after_model_callbacks - similarly, runs before/after the LLM model is called
on_model_error_callback - which runs when a model error is encountered

#### 3.2: To make things more concrete, what does a Plugin look like?

In [7]:
print("----- EXAMPLE PLUGIN - DOES NOTHING ----- ")

import logging
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.plugins.base_plugin import BasePlugin


# Applies to all agent and model calls
class CountInvocationPlugin(BasePlugin):
    """A custom plugin that counts agent and tool invocations."""

    def __init__(self) -> None:
        """Initialize the plugin with counters."""
        super().__init__(name="count_invocation")
        self.agent_count: int = 0
        self.tool_count: int = 0
        self.llm_request_count: int = 0

    # Callback 1: Runs before an agent is called. You can add any custom logic here.
    async def before_agent_callback(
        self, *, agent: BaseAgent, callback_context: CallbackContext
    ) -> None:
        """Count agent runs."""
        self.agent_count += 1
        logging.info(f"[Plugin] Agent run count: {self.agent_count}")

    # Callback 2: Runs before a model is called. You can add any custom logic here.
    async def before_model_callback(
        self, *, callback_context: CallbackContext, llm_request: LlmRequest
    ) -> None:
        """Count LLM requests."""
        self.llm_request_count += 1
        logging.info(f"[Plugin] LLM request count: {self.llm_request_count}")

----- EXAMPLE PLUGIN - DOES NOTHING ----- 


Key insight: You register a plugin once on your runner, and it automatically applies to every agent, tool call, and LLM request in your system as per your definition. 

#### 3.3: ADK's built-in LoggingPlugin
But you don't have to define all the callbacks and plugins to capture standard Observability data in ADK. Instead, ADK provides a built-in LoggingPlugin that automatically captures all agent activity:

🚀 User messages and agent responses
⏱️ Timing data for performance analysis
🧠 LLM requests and responses for debugging
🔧 Tool calls and results
✅ Complete execution traces

Agent definition
Let's use the same agent from the previous demo - the Research paper finder!

In [8]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search

from google.genai import types
from typing import List

retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)


def count_papers(papers: List[str]):
    """
    This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list.
    """
    return len(papers)


# Google search agent
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    description="Searches for information using Google search",
    instruction="Use the google_search tool to find information on the given topic. Return the raw search results.",
    tools=[google_search],
)

# Root agent
research_agent_with_plugin = LlmAgent(
    name="research_paper_finder_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Your task is to find research papers and count them. 
   
   You must follow these steps:
   1) Find research papers on the user provided topic using the 'google_search_agent'. 
   2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
   3) Return both the list of research papers and the total number of papers.
   """,
    tools=[AgentTool(agent=google_search_agent), count_papers],
)

print("✅ Agent created")

✅ Agent created


#### 3.4: Add LoggingPlugin to Runner
The following code creates the InMemoryRunner. This is used to programmatically invoke the agent.

To use LoggingPlugin in the above research agent,

Import the plugin
Add it when initializing the InMemoryRunner.

In [9]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=research_agent_with_plugin,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
)

print("✅ Runner configured")

plugin_manager.py:96 INFO: Plugin 'logging_plugin' registered.
runners.py:266 WARNING: App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "F:\AI-Agents-ADK\agents\Lib\site-packages\google\adk\agents", which implies app name "agents".


✅ Runner configured


Let's now run the agent using run_debug function.

In [10]:
print("🚀 Running agent with LoggingPlugin...")
print("📊 Watch the comprehensive logging output below:\n")

response = await runner.run_debug("Find recent papers on quantum computing")

🚀 Running agent with LoggingPlugin...
📊 Watch the comprehensive logging output below:


 ### Created new session: debug_session_id

User > Find recent papers on quantum computing
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-f37ce91d-1a1e-40de-bbea-febaf2654caa
[logging_plugin]    Session ID: debug_session_id
[logging_plugin]    User ID: debug_user_id
[logging_plugin]    App Name: InMemoryRunner
[logging_plugin]    Root Agent: research_paper_finder_agent
[logging_plugin]    User Content: text: 'Find recent papers on quantum computing'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-f37ce91d-1a1e-40de-bbea-febaf2654caa
[logging_plugin]    Starting Agent: research_paper_finder_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: research_paper_finder_agent
[logging_plugin]    Invocation ID: e-f37ce91d-1a1e-40de-bbea-febaf2654caa
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[l

google_llm.py:133 INFO: Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
google_llm.py:139 DEBUG: 
LLM Request:
-----------------------------------------------------------
System Instruction:
Your task is to find research papers and count them. 

   You must follow these steps:
   1) Find research papers on the user provided topic using the 'google_search_agent'. 
   2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
   3) Return both the list of research papers and the total number of papers.
   

You are an agent. Your internal name is "research_paper_finder_agent".
-----------------------------------------------------------
Config:
{'tools': [{}]}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"Find recent papers on quantum computing"}],"role":"user"}
-----------------------------------------------------------
Functions:
google_search_agent: {'request':

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: research_paper_finder_agent
[logging_plugin]    Content: function_call: google_search_agent
[logging_plugin]    Token Usage - Input: 240, Output: 21
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 0bd70e77-27e5-445a-be29-11e7828aaf97
[logging_plugin]    Author: research_paper_finder_agent
[logging_plugin]    Content: function_call: google_search_agent
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['google_search_agent']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: google_search_agent
[logging_plugin]    Agent: research_paper_finder_agent
[logging_plugin]    Function Call ID: adk-ba15bb7a-25cf-4401-ab9a-9066eb323f7d
[logging_plugin]    Arguments: {'request': 'recent papers on quantum computing'}
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-733c9687-56b5-4e8d-8c96-63818d152f70
[logging_plugin]    Session ID: 75dc21ed-7b4b-482b-89c2-

google_llm.py:133 INFO: Sending out request, model: gemini-2.5-flash-lite, backend: GoogleLLMVariant.GEMINI_API, stream: False
google_llm.py:139 DEBUG: 
LLM Request:
-----------------------------------------------------------
System Instruction:
Use the google_search tool to find information on the given topic. Return the raw search results.

You are an agent. Your internal name is "google_search_agent". The description about you is "Searches for information using Google search".
-----------------------------------------------------------
Config:
{}
-----------------------------------------------------------
Contents:
{"parts":[{"text":"recent papers on quantum computing"}],"role":"user"}
-----------------------------------------------------------
Functions:

-----------------------------------------------------------

models.py:6856 INFO: AFC is enabled with max remote calls: 10.
_trace.py:87 DEBUG: connect_tcp.started host='generativelanguage.googleapis.com' port=443 local_address=No

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: google_search_agent
[logging_plugin]    Content: text: 'Recent papers on quantum computing highlight significant advancements in hardware, algorithms, and error correction. Key developments include breakthroughs in qubit coherence times, with Princeton eng...'
[logging_plugin]    Token Usage - Input: 58, Output: 386
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 87c7bcac-5de1-445e-a88b-7939de7b5860
[logging_plugin]    Author: google_search_agent
[logging_plugin]    Content: text: 'Recent papers on quantum computing highlight significant advancements in hardware, algorithms, and error correction. Key developments include breakthroughs in qubit coherence times, with Princeton eng...'
[logging_plugin]    Final Response: True
[logging_plugin] 🤖 AGENT COMPLETED
[logging_plugin]    Agent Name: google_search_agent
[logging_plugin]    Invocation ID: e-733c9687-56b5-4e8d-8c96-63818d152f70
[logging_plugin] ✅ INVOCATION CO

_trace.py:87 DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Thu, 13 Nov 2025 07:10:39 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=1368'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
_client.py:1740 INFO: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
_trace.py:87 DEBUG: receive_response_body.started request=<Request [b'POST']>
_trace.py:87 DEBUG: receive_response_body.complete
_trace.py:87 DEBUG: response_closed.started
_trace.py:87 DEBUG: response_closed.complete
google_llm.py:186 INFO: Response receiv

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: research_paper_finder_agent
[logging_plugin]    Content: function_call: count_papers
[logging_plugin]    Token Usage - Input: 645, Output: 382
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 85dc109f-5933-4eee-b6ba-031ff981450a
[logging_plugin]    Author: research_paper_finder_agent
[logging_plugin]    Content: function_call: count_papers
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['count_papers']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: count_papers
[logging_plugin]    Agent: research_paper_finder_agent
[logging_plugin]    Function Call ID: adk-907cac99-219a-4432-b2a7-23696718dd5d
[logging_plugin]    Arguments: {'papers': ['Recent papers on quantum computing highlight significant advancements in hardware, algorithms, and error correction. Key developments include breakthroughs in qubit coherence times, with Princeton engineers creating a superconducting q

_trace.py:87 DEBUG: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=UTF-8'), (b'Vary', b'Origin'), (b'Vary', b'X-Origin'), (b'Vary', b'Referer'), (b'Content-Encoding', b'gzip'), (b'Date', b'Thu, 13 Nov 2025 07:10:41 GMT'), (b'Server', b'scaffolding on HTTPServer2'), (b'X-XSS-Protection', b'0'), (b'X-Frame-Options', b'SAMEORIGIN'), (b'X-Content-Type-Options', b'nosniff'), (b'Server-Timing', b'gfet4t7; dur=1707'), (b'Alt-Svc', b'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'), (b'Transfer-Encoding', b'chunked')])
_client.py:1740 INFO: HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
_trace.py:87 DEBUG: receive_response_body.started request=<Request [b'POST']>
_trace.py:87 DEBUG: receive_response_body.complete
_trace.py:87 DEBUG: response_closed.started
_trace.py:87 DEBUG: response_closed.complete
google_llm.py:186 INFO: Response receiv

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: research_paper_finder_agent
[logging_plugin]    Content: text: 'Here are the recent papers on quantum computing:

Recent papers on quantum computing highlight significant advancements in hardware, algorithms, and error correction. Key developments include breakthr...'
[logging_plugin]    Token Usage - Input: 1042, Output: 382
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 71d7efd3-85b4-4092-a117-a447231b0150
[logging_plugin]    Author: research_paper_finder_agent
[logging_plugin]    Content: text: 'Here are the recent papers on quantum computing:

Recent papers on quantum computing highlight significant advancements in hardware, algorithms, and error correction. Key developments include breakthr...'
[logging_plugin]    Final Response: True
research_paper_finder_agent > Here are the recent papers on quantum computing:

Recent papers on quantum computing highlight significant advancements in hardware, algorithms, 

### 📊 Summary
❓ When to use which type of Logging?

Development debugging? → Use adk web --log_level DEBUG
Common production observability? → Use LoggingPlugin()
Custom requirements? → Build Custom Callbacks and Plugins